# Using Flask

Flask is just a regular .py file that you run on the Anaconda prompt with `python script.py`.  When run, the script automatically starts a webserver as the script will contain a command calling a Flask app/object. 

URLs to refer:  
1. https://realpython.com/python-web-applications/  
2. https://towardsdatascience.com/python-webserver-with-flask-and-raspberry-pi-398423cc6f5d  
  
### First, standard pieces   
#### Top  
A Flask script file has some initial standard commands that import libraries, and initialize a Flask app.  
```python
from flask import Flask, request, escape, render_template
app = Flask(__name__)
```
#### Bottom
At the bottom of the script is a standard statement

```python
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=80, debug=True)
```
If you change the host to 127.0.0.1, then the webserver is only reachable on the localhost, and not the rest of the network.  To allow everyone on the network to access, use host=0.0.0.0.  Port can be set to anything.

### Second, the middle  

In the middle are a bunch of functions that get executed when a particular URL is called from any browser.  This particular URL is identified in the `@app.route(...)` command.  The below is 

```python
@app.route("/", methods = ["GET","POST"])
def index():
    return('something')  # 'something' then shows up in the browser.
    return render_template('index.html').
```
***  
#### The Details
The above is the simple part. Couple of additional things:  

You create two subdirectories under where the Python script is.  Called 'templates' and 'static'.  The first one you can't change the name, and you keep your html files there that are called by the different functions that are connected to URLs.  The second you use for keeping CSS files.  For CSS example see first example.  
  
In your function under the @app.route command, whatever you `return` will be displayed on the browser.  You could surround the text with html tags such as `<h1>...</h1>`.  Good enough for basic text.  
  
However to display more complex html, it is best to create a separate HTML file and call it using `render_template`.  In the HTML file, the dynamic components you keep as variables in the form `{{ variable1 }}`, just like in a mail-merge email.  These python will replace by real values of the variables before displaying the HTML.  The HTML files have to be kept under the 'templates' directory (directory has to be named exactly that).  You call the HTML file at the end of the function using `return render_template('something.html', **templateData)` where templateData is a dictionary with variables that are listed in the index.html file as `{{ variable_name }}`.  
  
Also, URLs can be dynamic, eg `"/<action>" ` where **action** could be *on* or *off* (which means the URL would be http://127.0.0.1/on etc).  Then the function can use the variable to do different things as below, see the GPIO example.
  
Another complication is getting data from users.  See second example.  The HTML file has the `<form>...</form>` for the fields.  The last one in this form grouping should be a submit button.  
  
**Difference between GET and POST:**  GET is used when the URL is called from the browser, and may contain the user supplied parameters appended to the end of the URL.  POST is when a submit button is clicked.  In the case of GET, you can extract the variables using, for example, `str(escape(request.args.get("celsius", "")))`.  For POST, you get variables using `request.form("variable_name")`.

#### Most basic example

This needs no HTML, no CSS, just the plain Python.  Note the second function with `"/<int:celsius>"`.  The angle bracket syntax (<>) tells Flask to capture any text following the base URL ("/") and pass it on to the function  as the variable celsius. Note that `fahrenheit_from()` function requires celsius as an input.  Adding `int:` before the variable name adds input validation, tells Flask to check whether the input it receives from the URL can be converted to an integer. If it can, then the content is passed on to fahrenheit_from(). If it can’t, then Flask displays a Not Found error page.  

So http://127.0.0.1/42 converts 42 celsius to fahrenheit.

```python
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return "Congratulations, it's a web app!"

@app.route("/<int:celsius>")
def fahrenheit_from(celsius):
    """Convert Celsius to Fahrenheit degrees."""
    fahrenheit = float(celsius) * 9 / 5 + 32
    fahrenheit = round(fahrenheit, 3)  # Round to three decimal places
    return str(fahrenheit)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=80, debug=True)
```
  
You can also insert a form right in the function itself without using a HTML file.  In the index function above, use the below for return.  

```python
@app.route("/")
def index():
    return """<form action="" method="get">
                <input type="text" name="celsius">
                <input type="submit" value="Convert">
              </form>"""
```



#### First example with HTML template

##### Controlling GPIO

In this example, going to a URL kicks off the function that changes the value of the GPIO pin and makes the LED on or off.  These URLs can be hyperlinked to make the 'buttons' for turning the LED on or off. (or hyperlinked in an image, which I didn't try.)  
  

**Python below **  
```python
import RPi.GPIO as GPIO
from flask import Flask, render_template
import time

app = Flask(__name__)

ledRed = 14
button=21
GPIO.setmode(GPIO.BCM)
GPIO.setup(ledRed, GPIO.OUT)
GPIO.output(ledRed, GPIO.LOW)
GPIO.setup(button, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)  


@app.route("/")
def index():
# Read Sensors Status
    ledRedSts = GPIO.input(ledRed)
    templateData = {
              'title' : 'GPIO output Status!',
              'ledRed'  : ledRedSts,
        }
    return render_template('index.html', **templateData)

@app.route("/<action>")
def action(action):
    if action == "on":
        GPIO.output(ledRed, GPIO.HIGH)
    if action == "off":
        GPIO.output(ledRed, GPIO.LOW)
     
    ledRedSts = GPIO.input(ledRed)
   
    templateData = {
              'ledRed'  : ledRedSts,
                    }
    return render_template('index.html', **templateData)

if __name__ == "__main__":
   app.run(host='0.0.0.0', port=80, debug=True)
```

**HTML as below: **  
```html
<!DOCTYPE html>
   <head>
      <title>GPIO Control</title>
      <link rel="stylesheet" href='../static/style.css'/>
   </head>
   <body>
		<h1>Actuators</h1>
		<h2> Status </h2>
		<h3> RED LED ==>  {{ ledRed  }}</h3>
		<br>
		<h2> Commands </h2>
		<h3> 
			RED LED Ctrl ==> 
			<a href="/on" class="button">TURN ON</a>  
			<a href="/off"class="button">TURN OFF</a>
		</h3>
				
   </body>
</html>
```

**CSS file:
```css
body {
   background: white;
   color: black;
}
```





#### Next example -  
##### Using forms to get user input to use in function

```python
from flask import Flask, request, escape, render_template
import datetime

app = Flask(__name__)

@app.route("/", methods = ["GET","POST"])
 
def hello():
    
    if request.method == "POST":
        celsius = request.form["celsius"]
        slidr = request.form["slidr"]
    else:
        celsius = 30 # start values so function doesn' error out
        slidr = 30 # start values
        
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M:%S")
    templateData = {
        'title' : 'HELLO!',
        'time': timeString,
        'slidr': slidr,
        'celsius': celsius
        }
    
    return render_template('index.html', **templateData)

```

For this file, the index.html looks like this:

```html
<!DOCTYPE html>
   <head>
      <title>{{ title }}</title>
	  <link rel="stylesheet" href=".\static\style.css">
   </head>
   <body>
      <h1>Hello, World!</h1>
      <h2>The date and time on the server is: {{ time }}</h2>
	  <h2>Slider value entered is: {{ celsius }}</h2>
	  <h2>Celsius entered is: {{ slidr }}</h2>
	  <form action="" method="POST">
                <input type="text" name="slidr" /> Slider Value<br>
                <input type="text" name="celsius" /> Celsius Value<br>
				<input type="submit" value="Submit" /> 
		</form>

   </body>
</html>
```


    
   
 